# Classificação de análise de fraude

A idéia principal do nosso projeto é analisar fraudes em solicitações de sisnistro feitas pelos dentistas que presta serviço para os segurados da odontoprev.

Ao realizar uma solicitação, o dentista deverá passar os sintomas, diagnóstico e procedimento que será realizado no segurado, assim como os insumos e quantidade que será usada para aquele procedimento.

Nossa inteligência artificial irá analisar e classificar essa solicitação como média, alta ou baixa chance de ser fraude.

## Importação e instalação das bibliotecas

Para esse projeto, vamos utilizar diversas bibliotecas e arquiteturas disponíveis para a linguagem python. Vou explicar cada uma delas

- **Transformers**: É uma arquitetura de rede neural artificial criada em 2017 por cientistas da Google Brain.

- **torch**: É uma framework usado para construir redes neurais combinado com biblioteca de aprendizado de máquina.

- **numpy**: Biblioteca que irá ajudar a fazer cálculos matemáticos.

- **pandas**: Biblioteca para analisar e manipular uma base de dados.

- **matplotlib.pyplot**: Biblioteca para criação de gráficos.

- **seaborn**: É uma biblioteca do Python que permite criar gráficos estatísticos para visualização de dados.

- **torch.nn**: Módulo do PyTorch fundamental para a construção e treinamento de redes neurais.

- **torch.optim**: Módulo do PyTorch que permite ajustar parâmetros do modelo para melhorar o mesmo.

- **train_test_split**: Responsável por separar os dados em treino e teste.

- **accuracy_score e classification_report**: Vaiser fundamental para avaliar a qualidade do modelo.

- **AutoTokenizer e AutoModel**: Facilita a criação de instância dos modelos disponíveis no hugging face.

In [ ]:
!pip install transformers

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModel

## Leitura e pré-processamento dos dados

In [ ]:
# Lendo o arquivo
dataframe = pd.read_csv('dados_dentista_large_dataset_150000_detailed.csv')
dataframe.sample(10)

,id_do_dentista,id_do_segurado,sintomas,diagnostico,procedimento,lista_de_insumos,quantidade_de_cada_insumo,chance_de_fraude
74835,2.0,664.0,Dor de dente,Pigmentação extrínseca,Placa miorrelaxante,Gel clareador; Moldeira,1; 3,0.0
136699,3.0,33.0,"Fratura parcial do dente incisivo superior, co...",Fratura dentária,Obturação,Resina composta; Adesivo dentinário,3; 1,0.0
14718,11.0,498.0,Dor de dente,Disfunção temporomandibular,Limpeza periodontal,Flúor; Escova dental,3; 3,0.0
63172,1.0,579.0,Dente quebrado,Disfunção temporomandibular,Obturação,Anestésico; Resina composta,1; 1,1.0
136147,3.0,94.0,Gengiva sangrando,Disfunção temporomandibular,Obturação,Flúor; Escova dental,2; 3,0.0
140104,2.0,113.0,Dor de dente,Fratura dentária,Placa miorrelaxante,Placa acrílica,1,2.0
118834,18.0,265.0,Dentes amarelados,Fratura dentária,Placa miorrelaxante,Placa acrílica,2,2.0
92379,10.0,381.0,Dor na mandíbula,Disfunção temporomandibular,Placa miorrelaxante,Gel clareador; Moldeira,3; 3,1.0
145603,1.0,597.0,Dente quebrado,Gengivite,Limpeza periodontal,Anestésico; Resina composta,2; 3,1.0
50514,4.0,519.0,Dentes amarelados,Gengivite,Placa miorrelaxante,Verniz fluoretado,1,2.0


In [ ]:
# Verificando as informações do dataset
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id_do_dentista             150000 non-null  float64
 1   id_do_segurado             150000 non-null  float64
 2   sintomas                   150000 non-null  object 
 3   diagnostico                150000 non-null  object 
 4   procedimento               150000 non-null  object 
 5   lista_de_insumos           150000 non-null  object 
 6   quantidade_de_cada_insumo  150000 non-null  object 
 7   chance_de_fraude           150000 non-null  float64
dtypes: float64(3), object(5)
memory usage: 9.2+ MB


### Detalhes importates do dataframe

- Possui 150 mil linhas e 8 colunas
- A maioria dos dados são do tipo object(string)
- O dataset não possui valores nulos
- A coluna target, ou seja, a que o nosso modelo vai predizer é a coluna 'chance_de_fraude'

Como os nossos dados variam muitos, transformar as colunas object no tipo numérico seria inviável.
Dessa forma, vamos ter que treinar nosso modelo usando um algoritmo que entenda descrições complexas como as que estão no nosso dataset.

Um bom exemplo seria usar Processamento de Linguagem Natural(NLP) pois com eles podemos transformar esses texto em dados vetorias com pesos e usar um modelo de NLP para processar esses dados.

## Tokenizer e modelo BERT

Mesmo que os modelos de NLP consigam entender descrições mais complexas, precisamos transformar os nossos dados textuais em dados numéricos pois os cálculos e ajustes de peso nos neurônios das redes neurais só funcionam com dados numéricos.

O tokenizer converte texto em uma sequência de tokens (valores numéricos que representam palavras ou subpalavras), e o model é a instância do modelo BERT.

BERT é um modelo de linguagem natural feito pelo google em 2018, ele é capaz de entender o contexto de uma palavra baseada em todas as palavras ao redor dela, antes ou depois.

Basicamente, a nossa função abaixo gera representações vetoriais (embeddings) para textos, que será útil para classificá-los posteriormente.

In [ ]:
# variável de ambiente que irá verificar se a gpu está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

# Carregar o tokenizer e o modelo e movê-lo para o dispositivo
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased").to(device)

# Função para gerar embeddings em lotes
def gerar_embeddings_lote(textos):
    # Mover inputs para o dispositivo (GPU se disponível)
    inputs = tokenizer(textos, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    # Mover embeddings de volta para a CPU para armazenamento no DataFrame
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embeddings

Usando o dispositivo: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## Transformando as colunas em dados vetoriais

Agora vamos usar a função acima para transformar as nossas colunas de texto em dados vetoriais, que futuramente iremos usar para treinar o nosso modelo.

In [ ]:
# Tratamento dos dados do tipo object

# como são muitas colunas vamos pegar somente metade do dataset
dataprocessado = dataframe.sample(100000)
# Processamento em lotes para cada coluna separada
batch_size = 16
for col in ['sintomas', 'diagnostico', 'procedimento']:
    embeddings = []
    for i in range(0, len(dataprocessado), batch_size):
        batch = dataprocessado[col].iloc[i:i+batch_size].tolist()
        embeddings_batch = gerar_embeddings_lote(batch)
        embeddings.extend(embeddings_batch)
    # Adiciona os embeddings em uma nova coluna
    dataprocessado[f'{col}_embedding'] = embeddings

dataprocessado.head()

,id_do_dentista,id_do_segurado,sintomas,diagnostico,procedimento,lista_de_insumos,quantidade_de_cada_insumo,chance_de_fraude,sintomas_embedding,diagnostico_embedding,procedimento_embedding
137178,19.0,626.0,Dor de dente,Pigmentação extrínseca,Clareamento dental,Gel clareador; Moldeira,1; 3,2.0,"[-0.3266826, 0.10571645, -0.03806349, 0.024753...","[-0.5930235, 0.04268049, -0.35204342, -0.13291...","[-0.15684341, 0.11068926, 0.0001742505, -0.241..."
20958,8.0,747.0,Dor de dente,Fratura dentária,Restauração,Bisturi; Sutura,3; 1,2.0,"[-0.3266826, 0.10571645, -0.03806349, 0.024753...","[-0.5079595, -0.28255412, 0.19769163, -0.22399...","[-0.46756575, 0.013842688, -0.15848188, -0.222..."
25111,20.0,469.0,Gengiva sangrando,Gengivite,Restauração,Resina composta; Adesivo dentinário,3; 1,2.0,"[-0.6839157, -0.32052204, 0.04164816, -0.42639...","[-0.45175576, 0.061760254, -0.17736633, -0.059...","[-0.46756575, 0.013842688, -0.15848188, -0.222..."
68818,13.0,979.0,Dor na mandíbula,"Disfunção na articulação temporomandibular, ca...",Obturação,Cureta; Antisséptico bucal,3; 2,2.0,"[-0.64675456, -0.08988595, -0.48830792, -0.257...","[-0.6257778, 0.45085144, -0.22975697, 0.089142...","[-0.62999797, -0.1930247, -0.15089221, -0.1339..."
86761,11.0,531.0,Gengiva sangrando,Cárie dentária,Clareamento dental,Verniz fluoretado,2,0.0,"[-0.6839157, -0.32052204, 0.04164816, -0.42639...","[-0.44253206, -0.023744138, -0.045338634, -0.1...","[-0.15684341, 0.11068926, 0.0001742505, -0.241..."


## Processamento dos insumos e sua quantidade

Abaixo fazemos o mesmo processo que fizemos anteriormente, transformamos os insumos em uma representação numérica (embedding) para que a máquina consiga processá-los.

O que fazemos de diferente é multiplicar cada embedding de insumo pela quantidade dele, isso significa que insumos com quantidades maiores terão mais influência na representação final.

Depois de obter todos os embedding ponderados, é calculado a média deles. Isso produz um único vetor que representa a combinação de todos os insumos e suas quantidades para aquela linha.


In [ ]:
# Tratamento da coluna insumos e quantidade

# Função para processar cada linha e gerar um embedding ponderado por quantidade
def processar_insumos_e_quantidades(row):
    insumos = row['lista_de_insumos'].split('; ')
    quantidades = list(map(int, row['quantidade_de_cada_insumo'].split('; ')))

    # Calcular o embedding ponderado por quantidade
    embeddings_ponderados = []
    for insumo, quantidade in zip(insumos, quantidades):
        embedding = gerar_embeddings_lote(insumo)[0]  # Gera o embedding do insumo
        embedding_ponderado = embedding * quantidade  # Multiplica o embedding pela quantidade
        embeddings_ponderados.append(embedding_ponderado)

    # Fazer a média dos embeddings ponderados para obter um vetor final representando a linha
    embedding_final = sum(embeddings_ponderados) / len(embeddings_ponderados)
    return embedding_final

# Aplicar a função e criar uma nova coluna com os pares
dataprocessado['insumos_quantidades'] = dataprocessado.apply(processar_insumos_e_quantidades, axis=1)

dataprocessado.head()

,id_do_dentista,id_do_segurado,sintomas,diagnostico,procedimento,lista_de_insumos,quantidade_de_cada_insumo,chance_de_fraude,sintomas_embedding,diagnostico_embedding,procedimento_embedding,insumos_quantidades
137178,19.0,626.0,Dor de dente,Pigmentação extrínseca,Clareamento dental,Gel clareador; Moldeira,1; 3,2.0,"[-0.3266826, 0.10571645, -0.03806349, 0.024753...","[-0.5930235, 0.04268049, -0.35204342, -0.13291...","[-0.15684341, 0.11068926, 0.0001742505, -0.241...","[-0.6604067, 0.02700691, -0.2291375, -0.047613..."
20958,8.0,747.0,Dor de dente,Fratura dentária,Restauração,Bisturi; Sutura,3; 1,2.0,"[-0.3266826, 0.10571645, -0.03806349, 0.024753...","[-0.5079595, -0.28255412, 0.19769163, -0.22399...","[-0.46756575, 0.013842688, -0.15848188, -0.222...","[-0.15148434, -0.35716096, -0.20276032, -0.091..."
25111,20.0,469.0,Gengiva sangrando,Gengivite,Restauração,Resina composta; Adesivo dentinário,3; 1,2.0,"[-0.6839157, -0.32052204, 0.04164816, -0.42639...","[-0.45175576, 0.061760254, -0.17736633, -0.059...","[-0.46756575, 0.013842688, -0.15848188, -0.222...","[-0.94715357, 0.31192768, -0.24945995, -0.5509..."
68818,13.0,979.0,Dor na mandíbula,"Disfunção na articulação temporomandibular, ca...",Obturação,Cureta; Antisséptico bucal,3; 2,2.0,"[-0.64675456, -0.08988595, -0.48830792, -0.257...","[-0.6257778, 0.45085144, -0.22975697, 0.089142...","[-0.62999797, -0.1930247, -0.15089221, -0.1339...","[-1.1535558, -0.008495986, -0.07700005, -0.580..."
86761,11.0,531.0,Gengiva sangrando,Cárie dentária,Clareamento dental,Verniz fluoretado,2,0.0,"[-0.6839157, -0.32052204, 0.04164816, -0.42639...","[-0.44253206, -0.023744138, -0.045338634, -0.1...","[-0.15684341, 0.11068926, 0.0001742505, -0.241...","[-1.159585, 0.2716215, 0.23807395, -0.03288733..."


In [ ]:
# Concatenar os embeddings em uma única lista de features
X = np.hstack([
    np.stack(dataprocessado['sintomas_embedding'].values),
    np.stack(dataprocessado['insumos_quantidades'].values),
    np.stack(dataprocessado['diagnostico_embedding'].values),
    np.stack(dataprocessado['procedimento_embedding'].values)
])
y = dataprocessado['chance_de_fraude'].values

# Separar o conjunto em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Transformar em tensores
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Definir uma rede neural mais potente (MLP)
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 3)  # 3 classes de saída
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.15)
        self.dropout3 = nn.Dropout(0.2)
        self.dropout4 = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.relu(self.fc4(x))
        x = self.fc5(x)  # Sem ReLU na camada de saída
        return x

# Inicializar o modelo, critério e otimizador
input_dim = X_train.shape[1]
model = MLP(input_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinar o modelo
epochs = 1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:  # Exibir o loss a cada 50 épocas
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Avaliar o modelo
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).argmax(dim=1)
    acc = accuracy_score(y_test_tensor, y_pred)
    print(f"Acurácia: {acc:.2f}")
    print(classification_report(y_test_tensor, y_pred))

Epoch 50/1000, Loss: 1.0900
Epoch 100/1000, Loss: 1.0494
Epoch 150/1000, Loss: 0.9993
Epoch 200/1000, Loss: 0.9553
Epoch 250/1000, Loss: 0.9200
Epoch 300/1000, Loss: 0.8842
Epoch 350/1000, Loss: 0.8649
Epoch 400/1000, Loss: 0.8457
Epoch 450/1000, Loss: 0.8305
Epoch 500/1000, Loss: 0.8121
Epoch 550/1000, Loss: 0.7975
Epoch 600/1000, Loss: 0.7892
Epoch 650/1000, Loss: 0.7844
Epoch 700/1000, Loss: 0.7742
Epoch 750/1000, Loss: 0.7683
Epoch 800/1000, Loss: 0.7607
Epoch 850/1000, Loss: 0.7572
Epoch 900/1000, Loss: 0.7512
Epoch 950/1000, Loss: 0.7483
Epoch 1000/1000, Loss: 0.7422
Acurácia: 0.69
              precision    recall  f1-score   support

           0       0.69      0.69      0.69     10080
           1       0.70      0.67      0.68     10073
           2       0.68      0.72      0.70      9847

    accuracy                           0.69     30000
   macro avg       0.69      0.69      0.69     30000
weighted avg       0.69      0.69      0.69     30000



## Conclusão


No nosso modelo inicial, focamos em principalmente pré-processar os dados para que o nosso modelo de processamento de linguagem natural (NLP) consiga aprender com eles.

De qualquer forma conseguimos atingir uma acurácia de 69% e, para as próximas entregas, o nosso foco é ajustar os parametros para aumentar essa nossa acurácia.